**Introduction to the Tkinter validation**

Tkinter validation relies on the three options that you can use for any input widget such as Entry widget:

- validate: specifies which type of event will trigger the validation.
- validatecommand: checks if a data is valid
- invalidcommand: executes when the data is invalid. In other words, it’ll execute if the validatecommand returns False.
- 
**validate**
The validate command can be one of the following string values:

# Tkinter Validation Modes

Tkinter provides several validation modes for widgets such as `Entry`. These modes determine when validation occurs.

| **Mode**     | **Description** |
|-------------|---------------|
| **`focus`**   | Validate whenever the widget gains or loses focus. |
| **`focusin`**  | Validate whenever the widget gains focus. |
| **`focusout`** | Validate whenever the widget loses focus. |
| **`key`**      | Validate whenever any keystroke changes the widget’s contents. |
| **`all`**      | Validate in all the above situations: `focusin`, `focusout`, and `key`. |
| **`none`**     | Turns validation off (default behavior). Note: The string `'none'` is different from the Python `None` value. |

### Example Usage:
```python```



In [1]:
import tkinter as tk

def validate_input(P):
    print(f"Validating: {P}")
    return True  # Always return True for testing

root = tk.Tk()
root.geometry("300x100")

vcmd = root.register(validate_input)  # Register validation function

entry = tk.Entry(root, validate="key", validatecommand=(vcmd, "%P"))
entry.pack(pady=20)

root.mainloop()

Validating: 1
Validating: 12
Validating: 124
Validating: 1245


**validatecommand**
  
The validatecommand is a tuple that contains:

- A reference to a Tcl/tk function.
- Zero or more substitution codes specify the information that triggers the event you want to pass into the function.
  
To get a reference to a Tck/tk function, you pass a callable to the widget.register() method. It returns a string that you can use with the validatecommand.

The following table shows the substitution codes that you can use with the tuple:

# Tkinter Validation Callback Substitutions

When using the `validatecommand` in Tkinter, special substitution codes can be used as arguments in the callback function. These provide useful information about the validation event.

| **Substitution Code** | **Description** |
|----------------------|---------------|
| **`%d`**  | Action code: `0` for deletion, `1` for insertion, `-1` for focus in/out or textvariable change. |
| **`%i`**  | Index of the beginning of the insertion or deletion. `-1` if triggered by focus in/out or textvariable change. |
| **`%P`**  | The new value of the widget if the change is allowed. |
| **`%s`**  | The current text in the entry before the change. |
| **`%S`**  | The text being inserted or deleted (if applicable). |
| **`%v`**  | The current value of the widget’s `validate` option. |
| **`%V`**  | The reason for this callback (`focusin`, `focusout`, `key`, or `forced` for textvariable change). |
| **`%W`**  | The Tkinter widget name triggering the callback. |

## Example Usage:

```python```


In [2]:
import tkinter as tk

def validate_input(action, index, new_value, old_value, text, validate, reason, widget_name):
    print(f"Action: {action}, Index: {index}, New: {new_value}, Old: {old_value}, Text: {text}, Validate: {validate}, Reason: {reason}, Widget: {widget_name}")
    return True  # Allow changes

root = tk.Tk()
root.geometry("300x100")

vcmd = root.register(validate_input)

entry = tk.Entry(root, validate="key", validatecommand=(vcmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
entry.pack(pady=20)

root.mainloop()


Action: 1, Index: 0, New: 4, Old: , Text: 4, Validate: key, Reason: key, Widget: .!entry
Action: 1, Index: 1, New: 45, Old: 4, Text: 5, Validate: key, Reason: key, Widget: .!entry
Action: 1, Index: 2, New: 456, Old: 45, Text: 6, Validate: key, Reason: key, Widget: .!entry
Action: 1, Index: 3, New: 4569, Old: 456, Text: 9, Validate: key, Reason: key, Widget: .!entry


The following example constructs a validatecommand that use the self.validate() method and %P substitution code:

`vcmd = (self.register(self.validate), '%P')`

**invalidcommand**

Like the validatecommand, the invalidcommand also requires the use of the widget.register() method and substitution code.

The following example returns a tuple that you can pass into the invalidcommand option:

`ivcmd = (self.register(self.on_invalid),)`

**Tkinter validation example**

We’ll create a form that contains an email input. If you enter an invalid email address, it’ll show an error message and change the text color of the email input to red. And we’ll trigger the validation event when the focus is moving out of the entry.

In [3]:
import tkinter as tk
from tkinter import ttk
import re


class App(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title('Tkinter Validation Demo')
        self.geometry("400x150")

        self.create_widgets()

    def create_widgets(self):
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=3)
        self.columnconfigure(2, weight=1)

        # Label
        ttk.Label(self, text='Email:').grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)

        # Validation command
        vcmd = (self.register(self.validate), '%P')
        ivcmd = (self.register(self.on_invalid),)

        # Email entry with validation
        self.email_var = tk.StringVar()
        self.email_entry = ttk.Entry(self, textvariable=self.email_var, width=40)
        self.email_entry.config(validate='focusout', validatecommand=vcmd, invalidcommand=ivcmd)
        self.email_entry.grid(row=0, column=1, padx=5, pady=5)

        # Error Label
        self.label_error = ttk.Label(self, foreground='red')
        self.label_error.grid(row=1, column=1, sticky=tk.W, padx=5)

        # Send Button
        self.send_button = ttk.Button(self, text='Send', command=self.submit)
        self.send_button.grid(row=0, column=2, padx=5)

    def show_message(self, error='', color='black'):
        self.label_error['text'] = error
        if color == 'red':
            self.email_entry.style = "Error.TEntry"
        else:
            self.email_entry.style = "TEntry"

    def validate(self, value):
        """
        Validate the email entry.
        """
        pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        if re.fullmatch(pattern, value) is None:
            return False

        self.show_message()
        return True

    def on_invalid(self):
        """
        Show the error message if the email is not valid.
        """
        self.show_message('Please enter a valid email', 'red')

    def submit(self):
        """
        Function to handle the Send button click.
        """
        if self.validate(self.email_var.get()):
            self.show_message("Email submitted!", "green")


if __name__ == '__main__':
    app = App()
    app.mainloop()
